# SQL approach

In [1]:
import pandas as pd

df = pd.read_csv("guada.csv", sep=';')
print(df.shape)
print(df.columns.tolist())

(70, 7)
['ID_PRODUCTO', 'STOCK', 'CATEGORÍA', 'REFERENCIA', 'COLOR', 'TALLA', 'PRECIO']


In [2]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///guada.db")
df.to_sql("guada", engine, index=False)

70

In [8]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM guada WHERE precio < 30000;")

sqlite
['guada']


"[(57, 2, 'Blusa Americana', 'Straple', 'Lila', 'Única', 25000), (58, 1, 'Blusa Americana', 'Straple', 'Salmón', 'Única', 25000), (59, 2, 'Blusa Americana', 'Boleros', 'Negro', 'Única', 25000), (60, 2, 'Blusa Americana', 'Boleros', 'Nude', 'Única', 25000), (61, 1, 'Blusa Americana', 'Boleros', 'Beige', 'Única', 25000), (62, 1, 'Blusa Americana', 'Cruzada ', 'Blanco', 'Única', 25000), (63, 1, 'Blusa Americana', 'Cruzada ', 'Nude', 'Única', 25000), (64, 1, 'Blusa Americana', 'Zizina', 'Rosa Pastel', 'Única', 25000), (65, 1, 'Blusa Americana', 'Zizina', 'Fuscia', 'Única', 25000), (66, 1, 'Blusa Americana', 'Dinive', 'Lila', 'Única', 25000), (67, 1, 'Blusa Americana', 'Dinive', 'Blanca', 'Única', 25000)]"

# Agente Taylor(ed)

### Ponele prompt y dale personalidad.

In [9]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
agent_executor = create_sql_agent(llm,
                                  db=db,
                                  agent_type="openai-tools",
                                  verbose=True
)

In [17]:
agent_executor.invoke({"input": "¿Cuál es el precio promedio de la categoría Blusa Americana?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


guada
Invoking: `sql_db_schema` with `{'table_names': 'guada'}`



CREATE TABLE guada (
	"ID_PRODUCTO" BIGINT, 
	"STOCK" BIGINT, 
	"CATEGORÍA" TEXT, 
	"REFERENCIA" TEXT, 
	"COLOR" TEXT, 
	"TALLA" TEXT, 
	"PRECIO" BIGINT
)

/*
3 rows from guada table:
ID_PRODUCTO	STOCK	CATEGORÍA	REFERENCIA	COLOR	TALLA	PRECIO
1	1	Sandalias	Morena 	Blanca	37	35000
2	1	Sandalias	Morena 	Blanca	38	35000
3	1	Sandalias	Morena 	Blanca	39	35000
*/
Invoking: `sql_db_query_checker` with `{'query': "SELECT AVG(PRECIO) as PrecioPromedio FROM guada WHERE CATEGORÍA = 'Blusa Americana'"}`


```sql
SELECT AVG(PRECIO) as PrecioPromedio FROM guada WHERE CATEGORÍA = 'Blusa Americana'
```
Invoking: `sql_db_query` with `{'query': "SELECT AVG(PRECIO) as PrecioPromedio FROM guada WHERE CATEGORÍA = 'Blusa Americana'"}`


[(25000.0,)]El precio promedio de la categoría "Blusa Americana" es de 25,000.0.

> Finished chain.


{'input': '¿Cuál es el precio promedio de la categoría Blusa Americana?',
 'output': 'El precio promedio de la categoría "Blusa Americana" es de 25,000.0.'}

In [12]:
agent_executor.invoke({"input": "¿Cuál es el producto más caro?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


guada
Invoking: `sql_db_schema` with `{'table_names': 'guada'}`



CREATE TABLE guada (
	"ID_PRODUCTO" BIGINT, 
	"STOCK" BIGINT, 
	"CATEGORÍA" TEXT, 
	"REFERENCIA" TEXT, 
	"COLOR" TEXT, 
	"TALLA" TEXT, 
	"PRECIO" BIGINT
)

/*
3 rows from guada table:
ID_PRODUCTO	STOCK	CATEGORÍA	REFERENCIA	COLOR	TALLA	PRECIO
1	1	Sandalias	Morena 	Blanca	37	35000
2	1	Sandalias	Morena 	Blanca	38	35000
3	1	Sandalias	Morena 	Blanca	39	35000
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT REFERENCIA, PRECIO FROM guada ORDER BY PRECIO DESC LIMIT 1;'}`


```sql
SELECT REFERENCIA, PRECIO FROM guada ORDER BY PRECIO DESC LIMIT 1;
```
Invoking: `sql_db_query` with `{'query': 'SELECT REFERENCIA, PRECIO FROM guada ORDER BY PRECIO DESC LIMIT 1;'}`


[('Morral Coach', 135000)]El producto más caro es el "Morral Coach" con un precio de 135,000.

> Finished chain.


{'input': '¿Cuál es el producto más caro?',
 'output': 'El producto más caro es el "Morral Coach" con un precio de 135,000.'}

In [16]:
agent_executor.invoke({"input": "¿Cuál es el precio de la categoría y referencia del producto más barato?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


guada
Invoking: `sql_db_schema` with `{'table_names': 'guada'}`



CREATE TABLE guada (
	"ID_PRODUCTO" BIGINT, 
	"STOCK" BIGINT, 
	"CATEGORÍA" TEXT, 
	"REFERENCIA" TEXT, 
	"COLOR" TEXT, 
	"TALLA" TEXT, 
	"PRECIO" BIGINT
)

/*
3 rows from guada table:
ID_PRODUCTO	STOCK	CATEGORÍA	REFERENCIA	COLOR	TALLA	PRECIO
1	1	Sandalias	Morena 	Blanca	37	35000
2	1	Sandalias	Morena 	Blanca	38	35000
3	1	Sandalias	Morena 	Blanca	39	35000
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT CATEGORÍA, REFERENCIA, PRECIO FROM guada ORDER BY PRECIO ASC LIMIT 1;'}`


```sql
SELECT CATEGORÍA, REFERENCIA, PRECIO FROM guada ORDER BY PRECIO ASC LIMIT 1;
```
Invoking: `sql_db_query` with `{'query': 'SELECT CATEGORÍA, REFERENCIA, PRECIO FROM guada ORDER BY PRECIO ASC LIMIT 1;'}`


[('Blusa Americana', 'Straple', 25000)]El producto más barato tiene un precio de 25,000, pertenece a la categoría "Blusa Americana" y su refe

{'input': '¿Cuál es el precio de la categoría y referencia del producto más barato?',
 'output': 'El producto más barato tiene un precio de 25,000, pertenece a la categoría "Blusa Americana" y su referencia es "Straple".'}

In [18]:
agent_executor.invoke({"input": "¿Cuáles son las sandalias más baratas?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


guada
Invoking: `sql_db_schema` with `{'table_names': 'guada'}`



CREATE TABLE guada (
	"ID_PRODUCTO" BIGINT, 
	"STOCK" BIGINT, 
	"CATEGORÍA" TEXT, 
	"REFERENCIA" TEXT, 
	"COLOR" TEXT, 
	"TALLA" TEXT, 
	"PRECIO" BIGINT
)

/*
3 rows from guada table:
ID_PRODUCTO	STOCK	CATEGORÍA	REFERENCIA	COLOR	TALLA	PRECIO
1	1	Sandalias	Morena 	Blanca	37	35000
2	1	Sandalias	Morena 	Blanca	38	35000
3	1	Sandalias	Morena 	Blanca	39	35000
*/
Invoking: `sql_db_query_checker` with `{'query': "SELECT REFERENCIA, COLOR, TALLA, PRECIO FROM guada WHERE CATEGORÍA = 'Sandalias' ORDER BY PRECIO ASC LIMIT 10;"}`


```sql
SELECT REFERENCIA, COLOR, TALLA, PRECIO FROM guada WHERE CATEGORÍA = 'Sandalias' ORDER BY PRECIO ASC LIMIT 10;
```
Invoking: `sql_db_query` with `{'query': "SELECT REFERENCIA, COLOR, TALLA, PRECIO FROM guada WHERE CATEGORÍA = 'Sandalias' ORDER BY PRECIO ASC LIMIT 10;"}`


[('Morena ', 'Blanca', '37', 35000), ('

{'input': '¿Cuáles son las sandalias más baratas?',
 'output': 'Las sandalias más baratas son las de la marca "Morena", con un precio de 35,000. Aquí tienes algunos ejemplos:\n\n1. **Referencia:** Morena, **Color:** Blanca, **Talla:** 37\n2. **Referencia:** Morena, **Color:** Blanca, **Talla:** 38\n3. **Referencia:** Morena, **Color:** Blanca, **Talla:** 39\n4. **Referencia:** Morena, **Color:** Blanca, **Talla:** 40\n5. **Referencia:** Morena, **Color:** Negro, **Talla:** 38\n6. **Referencia:** Morena, **Color:** Negro, **Talla:** 39\n7. **Referencia:** Morena, **Color:** Negro, **Talla:** 40\n8. **Referencia:** Morena, **Color:** Plata, **Talla:** 37\n9. **Referencia:** Morena, **Color:** Plata, **Talla:** 38\n10. **Referencia:** Morena, **Color:** Plata, **Talla:** 39\n\nTodos tienen el mismo precio de 35,000.'}

In [19]:
agent_executor.invoke({"input": "¿Qué productos puedo comprar con 50,000?"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


guada
Invoking: `sql_db_schema` with `{'table_names': 'guada'}`



CREATE TABLE guada (
	"ID_PRODUCTO" BIGINT, 
	"STOCK" BIGINT, 
	"CATEGORÍA" TEXT, 
	"REFERENCIA" TEXT, 
	"COLOR" TEXT, 
	"TALLA" TEXT, 
	"PRECIO" BIGINT
)

/*
3 rows from guada table:
ID_PRODUCTO	STOCK	CATEGORÍA	REFERENCIA	COLOR	TALLA	PRECIO
1	1	Sandalias	Morena 	Blanca	37	35000
2	1	Sandalias	Morena 	Blanca	38	35000
3	1	Sandalias	Morena 	Blanca	39	35000
*/
Invoking: `sql_db_query_checker` with `{'query': 'SELECT REFERENCIA, PRECIO FROM guada WHERE PRECIO <= 50000 ORDER BY PRECIO ASC LIMIT 10;'}`


```sql
SELECT REFERENCIA, PRECIO FROM guada WHERE PRECIO <= 50000 ORDER BY PRECIO ASC LIMIT 10;
```
Invoking: `sql_db_query` with `{'query': 'SELECT REFERENCIA, PRECIO FROM guada WHERE PRECIO <= 50000 ORDER BY PRECIO ASC LIMIT 10;'}`


[('Straple', 25000), ('Straple', 25000), ('Boleros', 25000), ('Boleros', 25000), ('Boleros', 25000), ('Cr

{'input': '¿Qué productos puedo comprar con 50,000?',
 'output': 'Con un presupuesto de 50,000, puedes comprar los siguientes productos:\n\n1. Straple - 25,000\n2. Boleros - 25,000\n3. Cruzada - 25,000\n4. Zizina - 25,000\n5. Dinive - 25,000\n\nTodos estos productos tienen un precio de 25,000, lo que te permite comprar múltiples unidades.'}